<a href="https://colab.research.google.com/github/RoboTuan/ML_DL_Project/blob/master/prova1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import os 
if not os.path.isdir('./ML_DL_Project'):
  !git clone https://github.com/RoboTuan/ML_DL_Project.git

Cloning into 'ML_DL_Project'...
remote: Enumerating objects: 94, done.
remote: Counting objects: 100% (94/94), done.
remote: Compressing objects: 100% (69/69), done.
remote: Total 94 (delta 56), reused 51 (delta 25), pack-reused 0
Unpacking objects: 100% (94/94), done.


In [0]:
# Se scritti uguali ai parametri della funzione può venire None come risultato
# per questo SEQLEN è maiuscolo 
SEQLEN = 7
NUM_CLASSES = 61
dataset = None
STAGE = None
train_data_dir = "./drive/My Drive/GTEA61"
val_data_dir = None
stage1_dict = None
out_dir = None
seqLen = None
trainBatchSize = 32
valBatchSize = None
numEpochs = 3
lr1 = 1e-3
decay_factor = None
decay_step = None
#Boh?!
MEMSIZE = 512

In [0]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Subset, DataLoader
from torch.backends import cudnn
import tensorflow as tf
import datetime
import numpy as np
import time
import matplotlib.pyplot as plt
import seaborn as sns
from torch.autograd import Function
import torchvision
from torchvision.models import alexnet
from torchvision.utils import make_grid
from sklearn.model_selection import RandomizedSearchCV
import pandas.util.testing as tm
from PIL import Image
from tqdm.notebook import tqdm

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [0]:
#!rm -rf "ML_DL_Project"

In [0]:
from __future__ import print_function, division
from ML_DL_Project.Scripts.objectAttentionModelConvLSTM import *
from ML_DL_Project.Scripts.spatial_transforms import (Compose, ToTensor, CenterCrop, Scale, Normalize, MultiScaleCornerCrop,
                                RandomHorizontalFlip)
from torch.utils.tensorboard import SummaryWriter
from ML_DL_Project.Scripts.makeDatasetRGB import *
import argparse
import sys

In [0]:
# Dir for saving models and log files
model_folder = os.path.join('./', "out_dir", "./GTEA61", 'rgb')

if not os.path.isdir('./out_dir'):
  if os.path.exists(model_folder):
    print('Directory {} exists!'.format(model_folder))
    sys.exit()
    
  os.makedirs(model_folder)

writer = SummaryWriter(model_folder)
train_log_loss = open((model_folder + '/train_log_loss.txt'), 'w')
train_log_acc = open((model_folder + '/train_log_acc.txt'), 'w')
val_log_loss = open((model_folder + '/val_log_loss.txt'), 'w')
val_log_acc = open((model_folder + '/val_log_acc.txt'), 'w')


In [0]:
# import os
# import torch
# from torch.utils.data import Dataset
# from PIL import Image
# import numpy as np
# import glob
# import random


# def gen_split(root_dir, stackSize):
#     Dataset = []
#     Labels = []
#     NumFrames = []
#     root_dir = os.path.join(root_dir, 'processed_frames2')
#     # print for debugging
#     #print(f"root_dir: {root_dir}")
#     for dir_user in sorted(os.listdir(root_dir)):
#       if not dir_user.startswith('.') and dir_user != "S2" :
#         class_id = 0
#         directory = os.path.join(root_dir, dir_user)
#         # print for debugging
#         #print(f"directory: {directory}")
#         for target in sorted(os.listdir(directory)):
#           if not target.startswith('.'):
#             directory1 = os.path.join(directory, target)
#             # print for debugging
#             #print(f"directory1: {directory1}")
#             insts = sorted(os.listdir(directory1))
#             # print for debugging
#             #print(f"insts: {insts}")
#             if insts != []:
#                for inst in insts:
#                  if not inst.startswith('.'):
#                    # adding "rgb" to path becasue after the number there are
#                    # both "rgb" and "mmap" directories
#                    inst = inst + "/rgb"
#                    inst_dir = os.path.join(directory1, inst)
#                    # print for debugging
#                    #print(f"inst_dir: {inst_dir}")
#                    numFrames = len(glob.glob1(inst_dir, '*.png'))
#                    # print for debugging
#                    #print(f"numFrames: {numFrames}")
#                    if numFrames >= stackSize:
#                      Dataset.append(inst_dir)
#                      Labels.append(class_id)
#                      NumFrames.append(numFrames)
#             class_id += 1
#     return Dataset, Labels, NumFrames

# class makeDataset(Dataset):
#     def __init__(self, root_dir, spatial_transform=None, seqLen=20,
#                  train=True, mulSeg=False, numSeg=1, fmt='.png'):

#         self.images, self.labels, self.numFrames = gen_split(root_dir, 5)
#         self.spatial_transform = spatial_transform
#         self.train = train
#         self.mulSeg = mulSeg
#         self.numSeg = numSeg
#         self.seqLen = seqLen
#         # print for debugging
#         #print(self.seqLen)
#         self.fmt = fmt

#     def __len__(self):
#         return len(self.images)

#     def __getitem__(self, idx):
#         vid_name = self.images[idx]
#         label = self.labels[idx]
#         numFrame = self.numFrames[idx]
#         inpSeq = []
#          # For debugging
#         #print(numFrame, self.seqLen)
#         self.spatial_transform.randomize_parameters()
#         for i in np.linspace(1, numFrame, self.seqLen, endpoint=False):
#           # Corrected with "rgb" instead of "image_" and zfill(4) instead of zfill(5)
#           fl_name = vid_name + '/' + 'rgb' + str(int(np.floor(i))).zfill(4) + self.fmt
#           img = Image.open(fl_name)
#           # For debugging
#           #print(img)
#           inpSeq.append(self.spatial_transform(img.convert('RGB')))
#         inpSeq = torch.stack(inpSeq, 0)
#         return inpSeq, label


In [0]:
# Data loader
normalize = Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
spatial_transform = Compose([Scale(256),
                             RandomHorizontalFlip(), 
                             MultiScaleCornerCrop([1, 0.875, 0.75, 0.65625], 224),
                             ToTensor(), 
                             normalize])

vid_seq_train = makeDataset(train_data_dir, spatial_transform=spatial_transform, seqLen=SEQLEN, fmt='.png')

train_loader = torch.utils.data.DataLoader(vid_seq_train, batch_size=trainBatchSize, shuffle=True, num_workers=4, pin_memory=True)

if val_data_dir is not None:
  vid_seq_val = makeDataset(val_data_dir,spatial_transform = Compose([Scale(256),
                                                                    CenterCrop(224),
                                                                    ToTensor(),
                                                                    normalize]),
                            seqLen=seqLen, fmt='.png')

  val_loader = torch.utils.data.DataLoader(vid_seq_val, batch_size=valBatchSize, shuffle=False, num_workers=2, pin_memory=True)
  valInstances = vid_seq_val.__len__()


trainInstances = vid_seq_train.__len__()

In [0]:
STAGE = 1

In [0]:
train_params = []
if STAGE == 1:

    model = attentionModel(num_classes=NUM_CLASSES, mem_size=MEMSIZE)
    model.train(False)
    for params in model.parameters():
        params.requires_grad = False
else:

    model = attentionModel(num_classes=NUM_CLASSES, mem_size=MEMSIZE)
    model.load_state_dict(torch.load(stage1_dict))
    model.train(False)
    for params in model.parameters():
        params.requires_grad = False
    #
    for params in model.resNet.layer4[0].conv1.parameters():
        params.requires_grad = True
        train_params += [params]

    for params in model.resNet.layer4[0].conv2.parameters():
        params.requires_grad = True
        train_params += [params]

    for params in model.resNet.layer4[1].conv1.parameters():
        params.requires_grad = True
        train_params += [params]

    for params in model.resNet.layer4[1].conv2.parameters():
        params.requires_grad = True
        train_params += [params]

    for params in model.resNet.layer4[2].conv1.parameters():
        params.requires_grad = True
        train_params += [params]
    #
    for params in model.resNet.layer4[2].conv2.parameters():
        params.requires_grad = True
        train_params += [params]
    #
    for params in model.resNet.fc.parameters():
        params.requires_grad = True
        train_params += [params]

    model.resNet.layer4[0].conv1.train(True)
    model.resNet.layer4[0].conv2.train(True)
    model.resNet.layer4[1].conv1.train(True)
    model.resNet.layer4[1].conv2.train(True)
    model.resNet.layer4[2].conv1.train(True)
    model.resNet.layer4[2].conv2.train(True)
    model.resNet.fc.train(True)

for params in model.lstm_cell.parameters():
    params.requires_grad = True
    train_params += [params]

for params in model.classifier.parameters():
    params.requires_grad = True
    train_params += [params]


In [0]:
model.lstm_cell.train(True)

model.classifier.train(True)
model.cuda()

loss_fn = nn.CrossEntropyLoss()

optimizer_fn = torch.optim.Adam(train_params, lr=lr1, weight_decay=4e-5, eps=1e-4)

optim_scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer_fn, milestones=decay_step, gamma=decay_factor)

train_iter = 0
min_accuracy = 0


for epoch in range(numEpochs):
    optim_scheduler.step()
    epoch_loss = 0
    numCorrTrain = 0
    trainSamples = 0
    iterPerEpoch = 0
    model.lstm_cell.train(True)
    model.classifier.train(True)
    writer.add_scalar('lr', optimizer_fn.param_groups[0]['lr'], epoch+1)
    if STAGE == 2:
        model.resNet.layer4[0].conv1.train(True)
        model.resNet.layer4[0].conv2.train(True)
        model.resNet.layer4[1].conv1.train(True)
        model.resNet.layer4[1].conv2.train(True)
        model.resNet.layer4[2].conv1.train(True)
        model.resNet.layer4[2].conv2.train(True)
        model.resNet.fc.train(True)
    for i, (inputs, targets) in enumerate(train_loader):
        train_iter += 1
        iterPerEpoch += 1
        optimizer_fn.zero_grad()
        inputVariable = Variable(inputs.permute(1, 0, 2, 3, 4).cuda())
        labelVariable = Variable(targets.cuda())
        trainSamples += inputs.size(0)
        output_label, _ = model(inputVariable)
        loss = loss_fn(output_label, labelVariable)
        loss.backward()
        optimizer_fn.step()
        _, predicted = torch.max(output_label.data, 1)
        numCorrTrain += (predicted == targets.cuda()).sum()
        epoch_loss += loss.data[0]
    avg_loss = epoch_loss/iterPerEpoch
    trainAccuracy = (numCorrTrain / trainSamples) * 100

    print('Train: Epoch = {} | Loss = {} | Accuracy = {}'.format(epoch+1, avg_loss, trainAccuracy))
    writer.add_scalar('train/epoch_loss', avg_loss, epoch+1)
    writer.add_scalar('train/accuracy', trainAccuracy, epoch+1)
    if val_data_dir is not None:
        if (epoch+1) % 1 == 0:
            model.train(False)
            val_loss_epoch = 0
            val_iter = 0
            val_samples = 0
            numCorr = 0
            for j, (inputs, targets) in enumerate(val_loader):
                val_iter += 1
                val_samples += inputs.size(0)
                inputVariable = Variable(inputs.permute(1, 0, 2, 3, 4).cuda(), volatile=True)
                labelVariable = Variable(targets.cuda(async=True), volatile=True)
                output_label, _ = model(inputVariable)
                val_loss = loss_fn(output_label, labelVariable)
                val_loss_epoch += val_loss.data[0]
                _, predicted = torch.max(output_label.data, 1)
                numCorr += (predicted == targets.cuda()).sum()
            val_accuracy = (numCorr / val_samples) * 100
            avg_val_loss = val_loss_epoch / val_iter
            print('Val: Epoch = {} | Loss {} | Accuracy = {}'.format(epoch + 1, avg_val_loss, val_accuracy))
            writer.add_scalar('val/epoch_loss', avg_val_loss, epoch + 1)
            writer.add_scalar('val/accuracy', val_accuracy, epoch + 1)
            val_log_loss.write('Val Loss after {} epochs = {}\n'.format(epoch + 1, avg_val_loss))
            val_log_acc.write('Val Accuracy after {} epochs = {}%\n'.format(epoch + 1, val_accuracy))
            if val_accuracy > min_accuracy:
                save_path_model = (model_folder + '/model_rgb_state_dict.pth')
                torch.save(model.state_dict(), save_path_model)
                min_accuracy = val_accuracy
        else:
            if (epoch+1) % 10 == 0:
                save_path_model = (model_folder + '/model_rgb_state_dict_epoch' + str(epoch+1) + '.pth')
                torch.save(model.state_dict(), save_path_model)

train_log_loss.close()
train_log_acc.close()
val_log_acc.close()
val_log_loss.close()
writer.export_scalars_to_json(model_folder + "/all_scalars.json")
writer.close()

/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:123: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)
/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1569: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1558: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")
Excep

IndexError: ignored